In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

In [2]:
data = pd.read_csv('kc_house_data.csv')

In [3]:
data.head()
list(data.columns)[3:]

['bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'waterfront',
 'view',
 'condition',
 'grade',
 'sqft_above',
 'sqft_basement',
 'yr_built',
 'yr_renovated',
 'zipcode',
 'lat',
 'long',
 'sqft_living15',
 'sqft_lot15']

In [4]:
class LinearRegressionS:
    def predict(self, X):
        return np.dot(X, self.W)
    def cost(self, Y_P, Y_T):
        return np.sum(np.square(Y_P - Y_T))/(2*Y_T.shape[0])
    def fit(self, X, Y, print_cost = False):
        self.W = np.linalg.pinv(X.transpose().dot(X)).dot(X.transpose()).dot(Y)

In [5]:
Y = pd.DataFrame(data['price']).to_numpy(copy = True)

In [6]:
temp = pd.DataFrame(data[list(data.columns)[3:]]).to_numpy(copy = True)

In [7]:
X = np.ones((temp.shape[0], temp.shape[1] + 1))

In [8]:
X[:, 1:] = temp

In [9]:
X_train = X[:15000]
Y_train = Y[:15000]
X_test = X[15000:]
Y_test = Y[15000:]

In [10]:
ls = LinearRegressionS()
ls.fit(X_train, Y_train, print_cost = False)

In [11]:
print("RMSE score :-" , np.sqrt(mean_squared_error(Y_test, ls.predict(X_test))))

RMSE score :- 203423.47568836645


In [12]:
print("R2 Score :-", r2_score(Y_test, ls.predict(X_test)))

R2 Score :- 0.6943982002041091


In [53]:
linreg = LinearRegression()
cross_val_score(linreg, temp[:15000], Y_train, cv=10, scoring = 'neg_root_mean_squared_error').mean()

-200823.12149870317

In [58]:
def cross_validation_split(X, Y, folds):
    dataset = np.concatenate((X, Y), axis = 1)
    np.random.shuffle(dataset)
    fold_size = len(X)//folds
    data_split = list()
    for i in range(folds):
        test_data = dataset[fold_size*(i):fold_size*(i+1), :]
        train_data = np.concatenate((dataset[:fold_size*(i), :], dataset[fold_size*(i+1):, :]), axis = 0)
        data_split.append([train_data, test_data])
    return data_split

In [59]:
def cross_validation(X, Y, folds):
    data_split = cross_validation_split(X, Y, folds)
    accuracy = list()
    for i in data_split:
        X_train = i[0][:, :-1]
        Y_train = i[0][:, -1]
        X_test = i[1][:, :-1]
        Y_test = i[1][:, -1]
        regressor = LinearRegressionS()
        regressor.fit(X_train, Y_train)
        accuracy.append(np.sqrt(mean_squared_error(Y_test, regressor.predict(X_test))))
    return np.array(accuracy)

In [60]:
cross_validation(X_train, Y_train, 10).mean()

201221.0323405717